In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.common.action_chains import ActionChains
# from selenium.webdriver.common.actions.whell_inpot import ScrollOrigin
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as soup
import time
import pandas as pd

출처 도시정비뉴스 / 재개발탭<br>
<a>https://www.dosijeongbi.com/sub.html?section=sc2</a>

In [135]:
df = pd.read_csv('data/서울시 부동산 실거래가 정보(2022~2025).csv', encoding='cp949')
# 강동구, 노원구, 성북구
gu_list = ['강동구', '노원구', '성북구', '서울', '그외']
gu_dict = {x: None for x in gu_list}
for gu in gu_list:
    gu_dict[gu] = {x : None for x in ['전체'] + sorted(df[df['자치구명']==gu]['법정동명'].unique().tolist())}
#청량리동은 동대문구임 잘못된 데이터
del gu_dict['성북구']['청량리동']

/var/folders/dy/qrk6x_0d333cl0bznn98dqbc0000gn/T/ipykernel_28025/2405103422.py:1: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/서울시 부동산 실거래가 정보(2022~2025).csv', encoding='cp949')


In [112]:
def crawling(news):
    
    news_list = []
    
    for i in range(10):
        headline = news[i].find('dt').text
        detail = news[i].find('dd', class_='sbody').text
        date = news[i].find('dd', class_='etc').text.strip().split('|')[-1].split()[0]

        news_list.append([headline, detail, date])
 
    return news_list

In [113]:
options = Options()
options.add_argument('--start-maximized')
options.add_argument('--disable-popup-blocking')

driver = webdriver.Chrome(
    options = options,
    service = ChromeService(ChromeDriverManager().install())
)

driver.get('https://www.dosijeongbi.com/sub.html?section=sc2')

i = 0
crawling_list = []
while True:
    
    driver.implicitly_wait(5)
    
    html = driver.page_source
    html_read = soup(html, 'lxml')
    
    news = html_read.find('div', id='sub_read_list').find_all('div',class_='sub_read_list_box')
    # check_detail(gu_dict, news)
    crawling_list+=crawling(news)
    # time.sleep(2)
    
    # break 조건: 2021년일시
    if news[-1].find('dd', class_='etc').text.strip().split('|')[-1].split()[0].split('.')[0] == '2021':
        print('2021년 데이터입니다 작동중지!')
        break
    
    next_button = driver.find_element(By.CSS_SELECTOR, 'div.paging').find_elements(By.CSS_SELECTOR, 'a')
    if i >= 9 :
        index = ((i+1)%10)+2
    else:
        index = i
    next_button[index].click()
    i+=1
    
    # time.sleep()
driver.quit()

2021년 데이터입니다 작동중지!


In [200]:
df_redevelopment_news = pd.DataFrame(crawling_list)
df_redevelopment_news.rename({0:'헤드라인', 1:'내용', 2:'날짜'}, axis=1, inplace =True) 


# df_redevelopment_news[df_redevelopment_news['헤드라인'].str.contains('|'.join(list(gu_dict.keys())[:-2]))]

for gu in list(gu_dict.keys())[::-1]:

    head_index = df_redevelopment_news[df_redevelopment_news['헤드라인'].str.contains(gu[:-1])].index
    body_index = df_redevelopment_news[df_redevelopment_news['내용'].str.contains(gu[:-1])].index

    df_redevelopment_news.loc[head_index, '자치구'] = gu
    df_redevelopment_news.loc[body_index, '자치구'] = gu
    
    for dong in list(gu_dict[gu].keys())[1:]:
        head_index2 = df_redevelopment_news[df_redevelopment_news['헤드라인'].str.contains(dong)].index
        body_index2 = df_redevelopment_news[df_redevelopment_news['내용'].str.contains(dong)].index
        # print(dong)
        df_redevelopment_news.loc[head_index2, '법정동'] = dong
        df_redevelopment_news.loc[body_index2, '법정동'] = dong
        
df_redevelopment_news['자치구'].fillna('그외',inplace = True)
df_redevelopment_news['법정동'].fillna('전체',inplace = True)
index = df_redevelopment_news[(df_redevelopment_news['법정동']=='길동')  & (df_redevelopment_news['자치구']=='서울')].index
df_redevelopment_news.loc[index,'법정동'] = '전체'

/var/folders/dy/qrk6x_0d333cl0bznn98dqbc0000gn/T/ipykernel_28025/4091450750.py:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_redevelopment_news['자치구'].fillna('그외',inplace = True)
/var/folders/dy/qrk6x_0d333cl0bznn98dqbc0000gn/T/ipykernel_28025/4091450750.py:23: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting v

In [204]:
# 총 139개의 데이터
df_redevelopment_news[df_redevelopment_news['자치구'].str.contains('|'.join(list(gu_dict.keys())[:-2]))]

,헤드라인,내용,날짜,자치구,법정동
10,"노원구 상계동 154-3번지 일대, 조합직접설립 추진",서울 노원구 상계동154-3번지 일대 주택정비형 재개발사업이 정비구역 지정 고시에 ...,2025.12.09,노원구,상계동
15,"성북1구역, GS-LH 손잡고 공공재개발...80m 단차 극복 특화설계",성북1구역이 GS건설과 LH의 손을 잡고 '자이 비원'으로 단지 생활의 시작을 알렸...,2025.12.05,성북구,전체
21,정릉·수유·답십리·중화2·창전·성현동 모아타운 지정..8478세대 주택 공급,서울시가 성북·강북·동대문·중랑·마포구 등 모아타운 6개 지역을 확정했다. 노후 저...,2025.12.03,성북구,전체
63,서울 강동역 역세권 활성화...818세대 복합단지로 탈바꿈,서울 5호선 강동역 일대에 총 818세대 규모의 공동주택과 판매 및 복지시설을 갖춘...,2025.10.27,강동구,전체
65,"노원구 상계5구역 재정비 현장 찾은 오세훈 ""인허가 절차·갈등해결에 힘 보탤 것""",서울시가 강북 지역 노후 주거지 재정비 사업 지원에 나선다. 오세훈 서울시장은 지난...,2025.10.25,노원구,전체
...,...,...,...,...,...
2155,재건축 최대어 '둔촌주공' 공사 중단 위기,서울 강동구 둔촌주공 재건축사업이 중단될 위기에 처했다. 시공사인 현대건설·HDC...,2022.03.16,강동구,전체
2165,"강동구 명일동 ‘삼익맨숀’, 합리적 재건축 위한 설명회 개최",서울 강동구 명일동 삼익맨숀(가든)이파트 재건축과 관련한 ‘빠르고 합리적인 재건축을...,2022.03.04,강동구,명일동
2167,"HDC현대산업개발, 월계동신아파트 재건축사업 수주",서울시 노원구 월계동신 아파트 주택재건축정비사업 시공사로 HDC현대산업개발이 선정됐...,2022.03.02,노원구,월계동
2170,강동구 명일동 삼익맨숀 분리 재건축 ‘이상동몽’ 솔로몬의 지혜는?,[취재 인터넷언론인연대 취재본부 편집 김아름내 기자] “토지분할 소송으로 5동을 제...,2022.03.01,강동구,명일동


In [209]:
df_redevelopment_news.to_csv('외부데이터/재개발뉴스.csv', encoding='utf-8')